**UAS TEMU KEMBALI INFORMASI**

- Jasmin Putri Jelita - 202110370311075
- Tazkia Damayanti - 202110370311090

# **Scraping**

In [2]:
import argparse
import io
import os
import re
import time
import urllib
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import date

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pdfminer import high_level
import concurrent.futures

In [3]:
!pip install pandas requests BeautifulSoup4 pdfminer.six lxml


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
def create_path(folder_name):
    path = os.path.join(os.getcwd(), folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def open_page(link):
    count = 0
    while count < 3:
        try:
            return BeautifulSoup(requests.get(link).text, "lxml")
        except:
            count += 1
            time.sleep(5)

def get_detail(soup, keyword):
    try:
        text = (
            soup.find(lambda tag: tag.name == "td" and keyword in tag.text)
            .find_next()
            .get_text()
            .strip()
        )
        return text
    except:
        return ""

def get_pdf(url, path_pdf):
    file = urllib.request.urlopen(url)
    file_name = file.info().get_filename().replace("/", " ")
    file_content = file.read()
    with open(f"{path_pdf}/{file_name}", "wb") as out_file:
        out_file.write(file_content)
    return io.BytesIO(file_content), file_name


def clean_text(text):
    
    replaced_patterns = [
        "M a h ka m a h A g u n g R e p u blik In d o n esia\n",
        "Disclaimer\n",
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n",
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n",
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n",
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n",
        "Direktori Putusan Mahkamah Agung Republik Indonesia\n",
        "putusan.mahkamahagung.go.id\n",
        "hkama ahkamah Agung Repub ahkamah Agung Republik Indonesia mah Agung Republik Indonesia blik Indonesi\n",
        "Disclaimer Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu. Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui : Email : kepaniteraan@mahkamahagung.go.id Telp : 021-384 3348 (ext.318)\n"
    ]
    
    for pattern in replaced_patterns:
        text = text.replace(pattern, "")

    text = ' '.join(text.replace('\n', ' ').split())
    text = re.sub(r'Halaman  \| \d+', '', text)
    text = re.sub(r'Halaman \d+ dari \d+', '', text)
    text = re.sub(r'Halaman \d+', '\n', text)

    return text


def save_pdf_text_to_txt(file_name_pdf, text_pdf, path_txt):
    today = date.today().strftime("%Y-%m-%d")
    txt_file_name = f"{file_name_pdf}_{today}.txt"
    with open(f"{path_txt}/{txt_file_name}", "w", encoding="utf-8") as txt_file:
        txt_file.write(text_pdf)

def extract_pdf_and_save(link_pdf, path_pdf, path_txt):
    try:
        # Download PDF and extract text
        file_pdf, file_name_pdf = get_pdf(link_pdf, path_pdf)
        text_pdf = high_level.extract_text(file_pdf)
        text_pdf = clean_text(text_pdf)
        
        # Save the PDF text to a .txt file
        save_pdf_text_to_txt(file_name_pdf, text_pdf, path_txt)
        return file_name_pdf, text_pdf  # Return the name and content of the PDF
    except Exception as e:
        print(f"Error extracting PDF from {link_pdf}: {e}")
        return "", ""

def extract_data(link, keyword_url):
    global today
    global path_output
    global path_pdf
    global path_txt

    path_output = create_path("csv_putusan_UAS")
    path_pdf = create_path("pdf_putusan_UAS")
    path_txt = create_path("txt_putusan_UAS")  # Directory for .txt files
    today = date.today().strftime("%Y-%m-%d")

    soup = open_page(link)
    table = soup.find("table", {"class": "table"})
    judul = table.find("h2").text
    soup.find("table", {"class": "table"}).find("h2").decompose()

    nomor = get_detail(table, "Nomor")
    tingkat_proses = get_detail(table, "Tingkat Proses")
    klasifikasi = get_detail(table, "Klasifikasi")
    kata_kunci = get_detail(table, "Kata Kunci")
    tahun = get_detail(table, "Tahun")
    tanggal_register = get_detail(table, "Tanggal Register")
    lembaga_peradilan = get_detail(table, "Lembaga Peradilan")
    jenis_lembaga_peradilan = get_detail(table, "Jenis Lembaga Peradilan")
    hakim_ketua = get_detail(table, "Hakim Ketua")
    hakim_anggota = get_detail(table, "Hakim Anggota")
    panitera = get_detail(table, "Panitera")
    amar = get_detail(table, "Amar")
    amar_lainnya = get_detail(table, "Amar Lainnya")
    catatan_amar = get_detail(table, "Catatan Amar")
    tanggal_musyawarah = get_detail(table, "Tanggal Musyawarah")
    tanggal_dibacakan = get_detail(table, "Tanggal Dibacakan")
    kaidah = get_detail(table, "Kaidah")
    abstrak = get_detail(table, "Abstrak")

    link_pdf = soup.find("a", href=re.compile(r"/pdf/"))["href"]
    
    # Use ThreadPoolExecutor to handle PDF extraction and saving in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(extract_pdf_and_save, link_pdf, path_pdf, path_txt)
        file_name_pdf, text_pdf = future.result()
        
    try:
        link_pdf = soup.find("a", href=re.compile(r"/pdf/"))["href"]
        file_pdf, file_name_pdf = get_pdf(link_pdf, path_pdf)
        text_pdf = high_level.extract_text(file_pdf)
        text_pdf = clean_text(text_pdf)
    except:
        link_pdf = ""
        text_pdf = ""
        file_name_pdf = ""


    # Collect all other data
    data = [
        judul,
        nomor,
        tingkat_proses,
        klasifikasi,
        kata_kunci,
        tahun,
        tanggal_register,
        lembaga_peradilan,
        jenis_lembaga_peradilan,
        hakim_ketua,
        hakim_anggota,
        panitera,
        amar,
        amar_lainnya,
        catatan_amar,
        tanggal_musyawarah,
        tanggal_dibacakan,
        kaidah,
        abstrak,
        link,
        link_pdf,
        file_name_pdf,
        text_pdf,
    ]

    result = pd.DataFrame(
        [data],
        columns=[
            "judul",
            "nomor",
            "tingkat_proses",
            "klasifikasi",
            "kata_kunci",
            "tahun",
            "tanggal_register",
            "lembaga_peradilan",
            "jenis_lembaga_peradilan",
            "hakim_ketua",
            "hakim_anggota",
            "panitera",
            "amar",
            "amar_lainnya",
            "catatan_amar",
            "tanggal_musyawarah",
            "tanggal_dibacakan",
            "kaidah",
            "abstrak",
            "link",
            "link_pdf",
            "file_name_pdf",
            "text_pdf",
        ],
    )

    # Create CSV filename with today's date and keyword URL
    keyword_url = keyword_url.replace("/", " ")
    if keyword_url.startswith("https"):
        keyword_url = ""

    destination = f"{path_output}/putusan_ma_{keyword_url}_{today}"
    print(destination)
    if not os.path.isfile(f"{destination}.csv"):
        result.to_csv(f"{destination}.csv", header=True, index=False)
    else:
        result.to_csv(f"{destination}.csv", mode="a", header=False, index=False)


In [7]:
#Dijalankan
def run_scraper(keyword=None, url=None, sort_date=True, download_pdf=True):
    if not keyword and not url:
        exit("Please provide keyword or URL")

    link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword}&page=1"

    if url:
        link = url

    soup = open_page(link)

    last_page = int(
        soup.find_all("a", {"class": "page-link"})[-1].get("data-ci-pagination-page")
    )

    if url:
        print(f"Scraping with url: {url} - {20 * last_page} data - {last_page} page")
    else:
        print(
            f"Scraping with keyword: {keyword} - {20 * last_page} data - {last_page} page"
        )

    if url:
        keyword_url = url
    else:
        keyword_url = keyword

    futures = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for page in range(last_page):
            futures.append(
                executor.submit(run_process, keyword_url, page + 1, sort_date)
            )
    wait(futures)

def run_process(keyword_url, page, sort_page):
    if keyword_url.startswith("https"):
        link = f"{keyword_url}&page={page}"
    else:
        link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword_url}&page={page}"
    if sort_page:
        link = f"{link}&obf=TANGGAL_PUTUS&obm=desc"

    print(link)

    soup = open_page(link)
    links = soup.find_all("a", {"href": re.compile("/direktori/putusan")})

    for link in links:
        extract_data(link["href"], keyword_url)

def scrape_specific_url(url, download_pdf=True):
  if not url or not url.startswith("https://"):
    print("please provide a valid URL")
    return

  extract_data(url,url)

In [ ]:
run_scraper(url="https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=099862PN172+++++++++++++++++++++&t_put=2024&t_reg=&t_upl=&t_pr=")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=099862PN172+++++++++++++++++++++&t_put=2024&t_reg=&t_upl=&t_pr= - 320 data - 16 page
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=099862PN172+++++++++++++++++++++&t_put=2024&t_reg=&t_upl=&t_pr=&page=1&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=099862PN172+++++++++++++++++++++&t_put=2024&t_reg=&t_upl=&t_pr=&page=2&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=099862PN172+++++++++++++++++++++&t_put=2024&t_reg=&t_upl=&t_pr=&page=3&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=&cat=3c40e48bbab311301a21c445b3c7fe57&jd=&tp=&court=099862PN172+++++++++++++++++++++

# **Overview**

In [1]:
import pandas as pd
import re

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
df = pd.read_csv('.\csv_putusan_UAS\putusan_ma__2025-01-04.csv')
df.head(5)

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,...,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,link,link_pdf,file_name_pdf,text_pdf
0,Putusan PN MATARAM Nomor 665/Pid.Sus/2024/PN M...,665/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,13 September 2024,PN MATARAM,PN,Hakim Ketua Dian Wicayanti,...,PIDANA PENJARA WAKTU TERTENTU,Menyatakan Terdakwa MUJAHIDIN Als JAHID Bin H....,20 Nopember 2024,20 Nopember 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_665_pid.sus_2024_pn_mtr_20250104023126...,\n P U T U S A NNomor 665/Pid.Sus/2024/PN MtrD...
1,Putusan PN MATARAM Nomor 719/Pid.Sus/2024/PN M...,719/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,10 Oktober 2024,PN MATARAM,PN,Hakim Ketua Kelik Trimargo,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI: Menyatakan Terdakwa Sahril Muttaqin...,18 Desember 2024,18 Desember 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_719_pid.sus_2024_pn_mtr_20250104023139...,\n P U T U S A NNomor 719/Pid.Sus/2024/PN MtrD...
2,Putusan PN MATARAM Nomor 742/Pid.Sus/2024/PN M...,742/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,21 Oktober 2024,PN MATARAM,PN,Hakim Ketua Isrin Surya Kurniasih,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I: Menyatakan Terdakwa I Gde S...,31 Desember 2024,31 Desember 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_742_pid.sus_2024_pn_mtr_20250104023144...,\n P U T U S A NNomor 742/Pid.Sus/2024/PN MtrD...
3,Putusan PN MATARAM Nomor 598/Pid.Sus/2024/PN M...,598/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,20 Agustus 2024,PN MATARAM,PN,Hakim Ketua Mukhlassuddin,...,PIDANA PENJARA WAKTU TERTENTU,Menyatakan terdakwa Pebrianto telah terbukti s...,2 Oktober 2024,2 Oktober 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_598_pid.sus_2024_pn_mtr_20250104023220...,\n P U T U S A NNomor 598/Pid.Sus/2024/PN MtrD...
4,Putusan PN MATARAM Nomor 655/Pid.Sus/2024/PN M...,655/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,11 September 2024,PN MATARAM,PN,Hakim Ketua Irlina,...,PIDANA PENJARA WAKTU TERTENTU,Menyatakan Terdakwa Nuzul Idris tersebut diata...,20 Nopember 2024,20 Nopember 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_655_pid.sus_2024_pn_mtr_20250104023238...,\n P U T U S A NNomor 655/Pid.Sus/2024/PN MtrD...


In [20]:
df.columns

Index(['judul', 'nomor', 'tingkat_proses', 'klasifikasi', 'kata_kunci',
       'tahun', 'tanggal_register', 'lembaga_peradilan',
       'jenis_lembaga_peradilan', 'hakim_ketua', 'hakim_anggota', 'panitera',
       'amar', 'amar_lainnya', 'catatan_amar', 'tanggal_musyawarah',
       'tanggal_dibacakan', 'kaidah', 'abstrak', 'link', 'link_pdf',
       'file_name_pdf', 'text_pdf'],
      dtype='object')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   judul                    74 non-null     object 
 1   nomor                    74 non-null     object 
 2   tingkat_proses           74 non-null     object 
 3   klasifikasi              74 non-null     object 
 4   kata_kunci               74 non-null     object 
 5   tahun                    74 non-null     int64  
 6   tanggal_register         74 non-null     object 
 7   lembaga_peradilan        74 non-null     object 
 8   jenis_lembaga_peradilan  74 non-null     object 
 9   hakim_ketua              74 non-null     object 
 10  hakim_anggota            74 non-null     object 
 11  panitera                 74 non-null     object 
 12  amar                     74 non-null     object 
 13  amar_lainnya             74 non-null     object 
 14  catatan_amar             74 

In [22]:
folder_path = 'pdf_putusan'

def read_pdf_filenames(folder_path):
    pdf_files = [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
    return pdf_files

pdf_filenames = read_pdf_filenames(folder_path)
all_angka_list = []

for filename in pdf_filenames:
    print(filename)
    match = re.search(r'putusan_(\d+)_pid', filename)

    if match:
        angka_str = match.group(1)[:50]
        angka_utuh = int(angka_str)
        angka_list = [angka_utuh]
        all_angka_list.append(angka_utuh)
        # print(angka_list)
    else:
        print("Tidak ada angka yang ditemukan antara kata 'putusan' dan 'pid' dalam nama file.")
print(all_angka_list)


putusan_249_pid.sus_2024_pn_mtr_20241105133208.pdf
putusan_251_pid.sus_2024_pn_mtr_20241105132742.pdf
putusan_261_pid.sus_2024_pn_mtr_20241105133107.pdf
putusan_268_pid.sus_2024_pn_mtr_20241105133153.pdf
putusan_299_pid.sus_2024_pn_mtr_20241105132651.pdf
putusan_321_pid.sus_2024_pn_mtr_20241105132840.pdf
putusan_340_pid.sus_2024_pn_mtr_20241105132912.pdf
putusan_349_pid.sus_2024_pn_mtr_20241105133022.pdf
putusan_359_pid.sus_2024_pn_mtr_20241105133437.pdf
putusan_374_pid.sus_2024_pn_mtr_20241105132431.pdf
putusan_375_pid.sus_2024_pn_mtr_20241105132746.pdf
putusan_378_pid.sus_2024_pn_mtr_20241105132205.pdf
putusan_379_pid.sus_2024_pn_mtr_20241105132315.pdf
putusan_386_pid.sus_2024_pn_mtr_20241105132254.pdf
putusan_390_pid.sus_2024_pn_mtr_20241105133013.pdf
putusan_404_pid.sus_2024_pn_mtr_20241105133119.pdf
putusan_410_pid.sus_2024_pn_mtr_20241105132701.pdf
putusan_411_pid.sus_2024_pn_mtr_20241105132557.pdf
putusan_413_pid.sus_2024_pn_mtr_20241105132722.pdf
putusan_424_pid.sus_2024_pn_mtr

In [23]:
df.duplicated().sum()
df = df.drop_duplicates()
df.reset_index(inplace=False)
df.head(5)

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,...,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,link,link_pdf,file_name_pdf,text_pdf
0,Putusan PN MATARAM Nomor 463/Pid.Sus/2024/PN M...,463/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,8 Juli 2024,PN MATARAM,PN,Hakim Ketua Mahyudin Igo,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I Menyatakan Terdakwa I GEDE M...,21 Agustus 2024,21 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,NaN,NaN,NaN
1,Putusan PN MATARAM Nomor 526/Pid.Sus/2024/PN M...,526/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,30 Juli 2024,PN MATARAM,PN,Hakim Ketua Mukhlassuddin,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI: Menyatakan Terdakwa LALU SONI IRAWA...,16 Oktober 2024,16 Oktober 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,NaN,NaN,NaN
2,Putusan PN MATARAM Nomor 482/Pid.Sus/2024/PN M...,482/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,15 Juli 2024,PN MATARAM,PN,Hakim Ketua Kelik Trimargo,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI: Menyatakan terdakwa WAHYU ALIAS TON...,21 Agustus 2024,21 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,NaN,NaN,NaN
3,Putusan PN MATARAM Nomor 568/Pid.Sus/2024/PN M...,568/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,12 Agustus 2024,PN MATARAM,PN,Hakim Ketua Mukhlassuddin,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI : Menyatakan Terdakwa PEBRIYAN RAMDA...,16 Oktober 2024,16 Oktober 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,NaN,NaN,NaN
4,Putusan PN MATARAM Nomor 510/Pid.Sus/2024/PN M...,510/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,24 Juli 2024,PN MATARAM,PN,Hakim Ketua Kelik Trimargo,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI:1. Menyatakan Terdakwa Jamiludin Akm...,11 September 2024,11 September 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,NaN,NaN,NaN


In [24]:
df = df[df['lembaga_peradilan'].str.contains('PN MATARAM', case=False, na=False)]
df= df.sort_values(by='nomor')
df.head()

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,...,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,link,link_pdf,file_name_pdf,text_pdf
61,Putusan PN MATARAM Nomor 249/Pid.Sus/2024/PN M...,249/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,3 April 2024,PN MATARAM,PN,Hakim Ketua Isrin Surya Kurniasih,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I: Menyatakan Terdakwa Burhanu...,22 Agustus 2024,22 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_249_pid.sus_2024_pn_mtr_20241105133208...,Direktori Putusan Mahkamah Agung Republik Indo...
39,Putusan PN MATARAM Nomor 251/Pid.Sus/2024/PN M...,251/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,3 April 2024,PN MATARAM,PN,Hakim Ketua I Ketut Somanasa,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI: Menyatakan Terdakwa Mustar Irawan A...,6 Agustus 2024,6 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_251_pid.sus_2024_pn_mtr_20241105132742...,Direktori Putusan Mahkamah Agung Republik Indo...
56,Putusan PN MATARAM Nomor 261/Pid.Sus/2024/PN M...,261/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,4 April 2024,PN MATARAM,PN,Hakim Ketua Isrin Surya Kurniasih,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I: Menyatakan Terdakwa Syarifu...,5 Agustus 2024,5 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_261_pid.sus_2024_pn_mtr_20241105133107...,Direktori Putusan Mahkamah Agung Republik Indo...
60,Putusan PN MATARAM Nomor 268/Pid.Sus/2024/PN M...,268/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,18 April 2024,PN MATARAM,PN,Hakim Ketua Isrin Surya Kurniasih,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I: Menyatakan Terdakwa terdakw...,1 Agustus 2024,1 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_268_pid.sus_2024_pn_mtr_20241105133153...,Direktori Putusan Mahkamah Agung Republik Indo...
34,Putusan PN MATARAM Nomor 299/Pid.Sus/2024/PN M...,299/Pid.Sus/2024/PN Mtr,Pertama,Pidana Khusus Narkotika dan Psikotropika \n P...,Narkotika,2024,8 Mei 2024,PN MATARAM,PN,Hakim Ketua I Ketut Somanasa,...,PIDANA PENJARA WAKTU TERTENTU,Menyatakan Terdakwa Sri Hikmawati Binti H. Bah...,7 Agustus 2024,7 Agustus 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_299_pid.sus_2024_pn_mtr_20241105132651...,Direktori Putusan Mahkamah Agung Republik Indo...


In [26]:
def save_results(path_output, keyword_url, current_date, data, file_name_pdf):
    result = pd.DataFrame([data])
    keyword_url = keyword_url.replace("/", " ")

    destination_csv = os.path.join(path_output, LOGGING_FILE_NAME)
    if not os.path.isfile(destination_csv):
        result.to_csv(destination_csv, header=True, index=False)
    else:
        result.to_csv(destination_csv, mode="a", header=False, index=False)

    destination_txt = os.path.join(path_output, f"{file_name_pdf}_{current_date}.txt")
    with open(destination_txt, 'w', encoding='utf-8') as file:
        file.write(data.get("text_pdf", ""))

In [11]:
df = df.loc[:, ['nomor', 'lembaga_peradilan','catatan_amar']]
df.head()

,nomor,lembaga_peradilan,catatan_amar
61,249/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa Burhanu...
39,251/Pid.Sus/2024/PN Mtr,PN MATARAM,MENGADILI: Menyatakan Terdakwa Mustar Irawan A...
56,261/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa Syarifu...
60,268/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa terdakw...
34,299/Pid.Sus/2024/PN Mtr,PN MATARAM,Menyatakan Terdakwa Sri Hikmawati Binti H. Bah...


In [12]:
def extract_barang_bukti(text):
    match = re.search(r"barang bukti berupa\s*(.*?)(;|berupa:\n)", text, re.DOTALL)
    if match:
        return match.group(1).strip()  
    else:
        return None  

df['barang bukti'] = df['catatan_amar'].apply(extract_barang_bukti)
df.head(50)

,nomor,lembaga_peradilan,catatan_amar,barang bukti
61,249/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa Burhanu...,: - 1 (satu) bungkus Narkotika jenis Shabu dib...
39,251/Pid.Sus/2024/PN Mtr,PN MATARAM,MENGADILI: Menyatakan Terdakwa Mustar Irawan A...,: 22 (dua puluh dua) bungkus plastik bening ya...
56,261/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa Syarifu...,: 1 (satu) buah plastic klip bening berisi kri...
60,268/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa terdakw...,: 1. 1 (satu) buah wadah berbentuk gelas warna...
34,299/Pid.Sus/2024/PN Mtr,PN MATARAM,Menyatakan Terdakwa Sri Hikmawati Binti H. Bah...,: - 1 (satu) dompet warna hitam
42,321/Pid.Sus/2024/PN Mtr,PN MATARAM,MENGADILIMenyatakan Terdakwa Saleh Obed Hamdun...,:30 (tiga puluh) plastic transparan yang didal...
47,340/Pid.Sus/2024/PN Mtr,PN MATARAM,MENGADILI:1. Menyatakan Terdakwa Fahrizal Bin ...,:- 1 (satu) celana pendek warna cokelat merk P...
52,349/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I: Menyatakan Terdakwa Sopha M...,: 1 (satu) plastic klip transparan yang didala...
70,359/Pid.Sus/2024/PN Mtr,PN MATARAM,MENGADILI: Menyatakan Para Terdakwa tersebut d...,: 1 (satu) klip plastik transparan yang didala...
19,374/Pid.Sus/2024/PN Mtr,PN MATARAM,M E N G A D I L I : Menyatakan Terdakwa I Indr...,: 1 (satu) buah bingkisan paket berwqarna hit...


In [13]:
df = df.dropna()
df = df[~df['barang bukti'].isnull() & (df['barang bukti'].str.strip() != '')]
df = df.rename(columns={'nomor': 'No Putusan'})
df = df.rename(columns={'lembaga_peradilan': 'Lembaga Peradilan'})
df = df.rename(columns={'barang bukti': 'Barang Bukti'})
df = df.rename(columns={'catatan_amar': 'Amar Putusan'})

In [14]:
df_final_summary = df.loc[:, ['No Putusan', 'Lembaga Peradilan','Barang Bukti','Amar Putusan']]
df_final_summary.head()

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan
61,249/Pid.Sus/2024/PN Mtr,PN MATARAM,: - 1 (satu) bungkus Narkotika jenis Shabu dib...,M E N G A D I L I: Menyatakan Terdakwa Burhanu...
39,251/Pid.Sus/2024/PN Mtr,PN MATARAM,: 22 (dua puluh dua) bungkus plastik bening ya...,MENGADILI: Menyatakan Terdakwa Mustar Irawan A...
56,261/Pid.Sus/2024/PN Mtr,PN MATARAM,: 1 (satu) buah plastic klip bening berisi kri...,M E N G A D I L I: Menyatakan Terdakwa Syarifu...
60,268/Pid.Sus/2024/PN Mtr,PN MATARAM,: 1. 1 (satu) buah wadah berbentuk gelas warna...,M E N G A D I L I: Menyatakan Terdakwa terdakw...
34,299/Pid.Sus/2024/PN Mtr,PN MATARAM,: - 1 (satu) dompet warna hitam,Menyatakan Terdakwa Sri Hikmawati Binti H. Bah...


In [15]:
df_final_summary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 61 to 26
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   No Putusan         64 non-null     object
 1   Lembaga Peradilan  64 non-null     object
 2   Barang Bukti       64 non-null     object
 3   Amar Putusan       64 non-null     object
dtypes: object(4)
memory usage: 2.5+ KB


In [17]:
df_final_summary.to_csv('data.csv', index=False)

print("File Excel berhasil disimpan.")

File Excel berhasil disimpan.
